In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gc

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

import numpy as np
import pickle

pd.options.display.max_columns = 500

def clip_y(y):
    return [min(y, 20) for y in y]

clip_y(np.array([1, 2, 3, 4, 56, 77, 20]))

[1, 2, 3, 4, 20, 20, 20]

In [2]:
df = pd.read_pickle("data/processed/sales_monthly_wide.pickle",)

In [ ]:
print(list(df.columns))

In [ ]:
df.head(2)

In [3]:
"""Let's reduce the dataset for a first tree model for simplicity"""

exclude_cols = ["item_category_name", "shop_name", "item_name"]
exclude_cols += [c for c in df.columns if "price" in c]
exclude_cols += [c for c in df.columns if "min" in c]

df_tree = df[[c for c in df.columns if c not in exclude_cols]].copy()

del df
gc.collect()

df_tree

,shop_id,item_id,item_category_id,sum_item_cnt_month_0,sum_item_cnt_month_1,sum_item_cnt_month_2,sum_item_cnt_month_3,sum_item_cnt_month_4,sum_item_cnt_month_5,sum_item_cnt_month_6,sum_item_cnt_month_7,sum_item_cnt_month_8,sum_item_cnt_month_9,sum_item_cnt_month_10,sum_item_cnt_month_11,sum_item_cnt_month_12,sum_item_cnt_month_13,sum_item_cnt_month_14,sum_item_cnt_month_15,sum_item_cnt_month_16,sum_item_cnt_month_17,sum_item_cnt_month_18,sum_item_cnt_month_19,sum_item_cnt_month_20,sum_item_cnt_month_21,sum_item_cnt_month_22,sum_item_cnt_month_23,sum_item_cnt_month_24,sum_item_cnt_month_25,sum_item_cnt_month_26,sum_item_cnt_month_27,sum_item_cnt_month_28,sum_item_cnt_month_29,sum_item_cnt_month_30,sum_item_cnt_month_31,sum_item_cnt_month_32,sum_item_cnt_month_33,max_item_cnt_month_0,max_item_cnt_month_1,max_item_cnt_month_2,max_item_cnt_month_3,max_item_cnt_month_4,max_item_cnt_month_5,max_item_cnt_month_6,max_item_cnt_month_7,max_item_cnt_month_8,max_item_cnt_month_9,max_item_cnt_month_10,max_item_cnt_month_11,max_item_cnt_month_12,max_item_cnt_month_13,max_item_cnt_month_14,max_item_cnt_month_15,max_item_cnt_month_16,max_item_cnt_month_17,max_item_cnt_month_18,max_item_cnt_month_19,max_item_cnt_month_20,max_item_cnt_month_21,max_item_cnt_month_22,max_item_cnt_month_23,max_item_cnt_month_24,max_item_cnt_month_25,max_item_cnt_month_26,max_item_cnt_month_27,max_item_cnt_month_28,max_item_cnt_month_29,max_item_cnt_month_30,max_item_cnt_month_31,max_item_cnt_month_32,max_item_cnt_month_33,max_date_0,max_date_1,max_date_2,max_date_3,max_date_4,max_date_5,max_date_6,max_date_7,max_date_8,max_date_9,max_date_10,max_date_11,max_date_12,max_date_13,max_date_14,max_date_15,max_date_16,max_date_17,max_date_18,max_date_19,max_date_20,max_date_21,max_date_22,max_date_23,max_date_24,max_date_25,max_date_26,max_date_27,max_date_28,max_date_29,max_date_30,max_date_31,max_date_32,max_date_33,count_date_0,count_date_1,count_date_2,count_date_3,count_date_4,count_date_5,count_date_6,count_date_7,count_date_8,count_date_9,count_date_10,count_date_11,count_date_12,count_date_13,count_date_14,count_date_15,count_date_16,count_date_17,count_date_18,count_date_19,count_date_20,count_date_21,count_date_22,count_date_23,count_date_24,count_date_25,count_date_26,count_date_27,count_date_28,count_date_29,count_date_30,count_date_31,count_date_32,count_date_33
0,0,30,40,0,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,31,37,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,32,40,6,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,33,37,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,35,40,1,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# Decision Tree

## Initial observation with a non time series setup (not usable for november prediciton)

First we start with a standard model, which is not adjusted to thew time series nature of the problem,
i.e. we do a vertical split. This results easily in an rmse of < 0.8, which would be an outstanding result in the real competition.

In [ ]:
df_tree_X = df_tree.set_index(["shop_id", "item_id"])
df_tree_y = df_tree_X["sum_item_cnt_month_33"]

for col in df_tree_X.columns:
    if "33" in col:
        print(f"Removing column {col}")
        del df_tree_X[col]
    
X_train, X_test, y_train, y_test = train_test_split(df_tree_X, df_tree_y, test_size=0.2, random_state=42)

X_train

In [ ]:
model_tree = DecisionTreeRegressor(
    max_depth=10,
    min_samples_leaf=100,
)

model_tree.fit(X_train, y_train)

In [ ]:
y_pred_test = model_tree.predict(X_test)
y_pred_train = model_tree.predict(X_train)

print(f"""
Error on test: {mean_squared_error(y_true=clip_y(y_test), y_pred=clip_y(y_pred_test), squared=False)}
Error on train: {mean_squared_error(y_true=clip_y(y_train), y_pred=clip_y(y_pred_train), squared=False)}
""")

In [ ]:
del df_tree_X, df_tree_y, X_train, X_test, y_train, y_test, y_pred_test, y_pred_train, model_tree

gc.collect()

## predicting via time series structure

In [ ]:
df_tree

In [ ]:
"""We will create a new dataframe that has all possible sub timeseries of length k
which we will user for training a model"""

k = 18

for i in tqdm(range(34 - k)):
    col_index_list = [str(j) for j in range(i, i + k + 1)]
    cols = ["shop_id", "item_id", "item_category_id"]
    cols += [col for col in df_tree.columns if col.split("_")[-1] in col_index_list]
    
    df_temp = df_tree[cols].copy()
    

    if i == 0:
        df_tree_series = df_temp.copy()
        
    else:
        df_temp.columns = df_tree_series.columns
        df_tree_series = df_tree_series.append(df_temp)
        
df_tree_series = df_tree_series.reset_index(drop=True)

del df_temp, cols
gc.collect()

In [ ]:
df_tree_series

In [ ]:
df_tree_series.info()

In [ ]:
del df_tree
gc.collect()

In [ ]:
df_tree_series = df_tree_series.sample(frac=1).reset_index(drop=True)

In [ ]:
df_tree_series

In [ ]:
df_tree_series.to_pickle("data/processed/df_tree_series.pickle")

In [ ]:
"""memory management doe snot really work..."""

df_tree_series = pd.read_pickle("data/processed/df_tree_series.pickle")

In [ ]:
df_tree_series

In [ ]:
import math

y = df_tree_series["sum_item_cnt_month_18"]

for col in df_tree_series:
    if "18" in col:
        del df_tree_series[col]
        
split_index = math.floor(df_tree_series.shape[0] * 0.7)

split_index

In [ ]:
model_tree_series = DecisionTreeRegressor(
    max_depth=3
)

model_tree_series.fit(
    df_tree_series.iloc[:split_index,:],
    y[:split_index]
)

In [ ]:
print(f"""
Error on train: {
    mean_squared_error(
        y_true=clip_y(y[:split_index]),
        y_pred=clip_y(model_tree_series.predict(df_tree_series.iloc[:split_index,:])),
        squared=False
    )
}
Error on test: {
    mean_squared_error(
        y_true=clip_y(y[split_index:]),
        y_pred=clip_y(model_tree_series.predict(df_tree_series.iloc[split_index:,:])),
        squared=False
    )
}
""")

In [ ]:
with open('data/models/model_tree_series.pickle', 'wb') as file:
    pickle.dump(model_tree_series, file)   

In [4]:
with open('data/models/model_tree_series.pickle', 'rb') as model:
    model_tree_series = pickle.load(model)

In [5]:
model_tree_series

DecisionTreeRegressor(max_depth=3)

In [6]:
del df_tree_series

NameError: name 'df_tree_series' is not defined

In [ ]:
gc.collect()

In [7]:
"""Create dataset for predicting on"""
k = 18

cols = ["shop_id", "item_id", "item_category_id"]
col_index_list = [str(x) for x in range(34 - k, 34)]

cols += [col for col in df_tree.columns if col.split("_")[-1] in col_index_list]
    
df_to_predict = df_tree[cols]

df_to_predict

,shop_id,item_id,item_category_id,sum_item_cnt_month_16,sum_item_cnt_month_17,sum_item_cnt_month_18,sum_item_cnt_month_19,sum_item_cnt_month_20,sum_item_cnt_month_21,sum_item_cnt_month_22,sum_item_cnt_month_23,sum_item_cnt_month_24,sum_item_cnt_month_25,sum_item_cnt_month_26,sum_item_cnt_month_27,sum_item_cnt_month_28,sum_item_cnt_month_29,sum_item_cnt_month_30,sum_item_cnt_month_31,sum_item_cnt_month_32,sum_item_cnt_month_33,max_item_cnt_month_16,max_item_cnt_month_17,max_item_cnt_month_18,max_item_cnt_month_19,max_item_cnt_month_20,max_item_cnt_month_21,max_item_cnt_month_22,max_item_cnt_month_23,max_item_cnt_month_24,max_item_cnt_month_25,max_item_cnt_month_26,max_item_cnt_month_27,max_item_cnt_month_28,max_item_cnt_month_29,max_item_cnt_month_30,max_item_cnt_month_31,max_item_cnt_month_32,max_item_cnt_month_33,max_date_16,max_date_17,max_date_18,max_date_19,max_date_20,max_date_21,max_date_22,max_date_23,max_date_24,max_date_25,max_date_26,max_date_27,max_date_28,max_date_29,max_date_30,max_date_31,max_date_32,max_date_33,count_date_16,count_date_17,count_date_18,count_date_19,count_date_20,count_date_21,count_date_22,count_date_23,count_date_24,count_date_25,count_date_26,count_date_27,count_date_28,count_date_29,count_date_30,count_date_31,count_date_32,count_date_33
0,0,30,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,31,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,32,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,33,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,35,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,22154,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
424120,59,22155,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
424121,59,22162,40,0,0,0,0,0,0,0,0,0,9,4,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,24,26,17,29,0,0,11,0,0,0,0,0,0,0,0,0,0,0,9,4,1,1,0,0,1,0,0
424122,59,22164,37,0,0,0,0,0,0,0,0,0,2,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,13,29,30,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,0,1,0,0,0


In [8]:
preds = clip_y(model_tree_series.predict(df_to_predict))

In [9]:
df_to_predict["pred"] = preds

<ipython-input-9-216fb4233b04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_predict["pred"] = preds


In [10]:
df_to_predict

,shop_id,item_id,item_category_id,sum_item_cnt_month_16,sum_item_cnt_month_17,sum_item_cnt_month_18,sum_item_cnt_month_19,sum_item_cnt_month_20,sum_item_cnt_month_21,sum_item_cnt_month_22,sum_item_cnt_month_23,sum_item_cnt_month_24,sum_item_cnt_month_25,sum_item_cnt_month_26,sum_item_cnt_month_27,sum_item_cnt_month_28,sum_item_cnt_month_29,sum_item_cnt_month_30,sum_item_cnt_month_31,sum_item_cnt_month_32,sum_item_cnt_month_33,max_item_cnt_month_16,max_item_cnt_month_17,max_item_cnt_month_18,max_item_cnt_month_19,max_item_cnt_month_20,max_item_cnt_month_21,max_item_cnt_month_22,max_item_cnt_month_23,max_item_cnt_month_24,max_item_cnt_month_25,max_item_cnt_month_26,max_item_cnt_month_27,max_item_cnt_month_28,max_item_cnt_month_29,max_item_cnt_month_30,max_item_cnt_month_31,max_item_cnt_month_32,max_item_cnt_month_33,max_date_16,max_date_17,max_date_18,max_date_19,max_date_20,max_date_21,max_date_22,max_date_23,max_date_24,max_date_25,max_date_26,max_date_27,max_date_28,max_date_29,max_date_30,max_date_31,max_date_32,max_date_33,count_date_16,count_date_17,count_date_18,count_date_19,count_date_20,count_date_21,count_date_22,count_date_23,count_date_24,count_date_25,count_date_26,count_date_27,count_date_28,count_date_29,count_date_30,count_date_31,count_date_32,count_date_33,pred
0,0,30,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
1,0,31,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
2,0,32,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
3,0,33,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
4,0,35,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,22154,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
424120,59,22155,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.168993
424121,59,22162,40,0,0,0,0,0,0,0,0,0,9,4,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,24,26,17,29,0,0,11,0,0,0,0,0,0,0,0,0,0,0,9,4,1,1,0,0,1,0,0,0.168993
424122,59,22164,37,0,0,0,0,0,0,0,0,0,2,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,13,29,30,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,2,1,2,0,0,1,0,0,0,0.168993


In [11]:
submission = df_to_predict[["shop_id", "item_id", "pred"]]

submission

,shop_id,item_id,pred
0,0,30,0.168993
1,0,31,0.168993
2,0,32,0.168993
3,0,33,0.168993
4,0,35,0.168993
...,...,...,...
424119,59,22154,0.168993
424120,59,22155,0.168993
424121,59,22162,0.168993
424122,59,22164,0.168993


In [12]:
pd.read_csv("data/source/sample_submission.csv")

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [13]:
test = pd.read_csv("data/source/test.csv")

test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [22]:
merged = submission.join(test.set_index(["shop_id", "item_id"]), how="right", on=["shop_id", "item_id"])

merged

,shop_id,item_id,pred,ID
24972,5,5037,0.168993,0
424123,5,5320,NaN,1
25008,5,5233,0.168993,2
25007,5,5232,0.168993,3
424123,5,5268,NaN,4
...,...,...,...,...
309302,45,18454,0.168993,214195
424123,45,16188,NaN,214196
308318,45,15757,0.168993,214197
424123,45,19648,NaN,214198


In [23]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 24972 to 303288
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   shop_id  214200 non-null  int64  
 1   item_id  214200 non-null  int64  
 2   pred     111404 non-null  float64
 3   ID       214200 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 8.2 MB


In [26]:
submission = merged[["ID", "pred"]].reset_index(drop=True)

submission.columns = ["ID", "item_cnt_month"]

submission = submission.fillna(0)

submission

,ID,item_cnt_month
0,0,0.168993
1,1,0.000000
2,2,0.168993
3,3,0.168993
4,4,0.000000
...,...,...
214195,214195,0.168993
214196,214196,0.000000
214197,214197,0.168993
214198,214198,0.000000


In [28]:
submission.to_csv("data/submission_decision_tree1.csv", index=False)

In [29]:
submission.sort_values(by="item_cnt_month")

,ID,item_cnt_month
175158,175158,0.0
172607,172607,0.0
172608,172608,0.0
172609,172609,0.0
172610,172610,0.0
...,...,...
37296,37296,20.0
25960,25960,20.0
41260,41260,20.0
138160,138160,20.0
